In [2]:
import numpy as np
import pandas as pd

In [12]:
# Parametros para função de mobilidade
MD = 20 # Distância máxima do salto - variações 0=nenhuma 5,10 e 20=Todo o grid 
IM = 20 # Proporção de individuos móveis - variações: 0,20%,50% e 100% 

rows, cols = 20, 20
n_cells = rows * cols  

# Estados possíveis 0=S, 1=E, 2=I, 3=R, 4=Q 
# Cria o dataframe com a população e seus Parametros individuais
data = {
    'status': np.concatenate([np.zeros(n_cells*95//100), np.ones(n_cells*5//100) * 2]).astype(int),   # 95% da populacao é S e 5% é I
    'learningCapability': np.random.uniform(0.5, 1.5, n_cells),  # aleatorio entre 0.5 e 1.5
    'transferCapability': np.random.uniform(0.5, 1.5, n_cells),  # aleatorio entre 0.5 e 1.5
    'baseForgettingRate': np.random.uniform(0.1, 0.3, n_cells),  # aleatorio entre 0.1 e 0.3
    'baseQuittingRate': np.random.uniform(0.1, 0.3, n_cells),    # aleatorio entre 0.1 e 0.3
    'timesContactorD': np.zeros(n_cells)
}
# Create row and column indices to represent the 20x20 grid
row_indices, col_indices = np.divmod(np.arange(n_cells), cols)
row_indices = row_indices.astype(int)
col_indices = col_indices.astype(int)

# Create DataFrame
cellularGrid = pd.DataFrame(data)
cellularGrid['row'] = row_indices
cellularGrid['col'] = col_indices

# columns to have row and col first
cellularGrid = cellularGrid[['row', 'col', 'status', 'learningCapability', 'transferCapability', 
                             'baseForgettingRate', 'baseQuittingRate', 'timesContactorD']]

# Display the first few rows
print(cellularGrid.head())

   row  col  status  learningCapability  transferCapability  \
0    0    0       0            1.276506            1.498387   
1    0    1       0            1.455467            0.944534   
2    0    2       0            0.659490            1.435620   
3    0    3       0            0.769684            1.243863   
4    0    4       0            1.026490            0.740193   

   baseForgettingRate  baseQuittingRate  timesContactorD  
0            0.101057          0.169590              0.0  
1            0.269587          0.294270              0.0  
2            0.103217          0.220986              0.0  
3            0.129957          0.269436              0.0  
4            0.125507          0.294131              0.0  


In [14]:
# Verify the shape and status distribution
print("\nDataFrame shape:", cellularGrid.shape)
print("\nStatus distribution:")
print(cellularGrid['status'].value_counts())


DataFrame shape: (400, 8)

Status distribution:
status
0    380
2     20
Name: count, dtype: int64


In [15]:
def mobility(df, MD, IM, n_cells):
    # Copy DataFrame to avoid modifying the original
    new_df = df.copy()
    
    # Calculate MDGrid based on MD
    if MD == 0:
        MDGrid = 0
    elif MD == 5:
        MDGrid = (n_cells * 32) // 100  # 32% of gridSize
    elif MD == 10:
        MDGrid = (n_cells * 64) // 100  # 64% of gridSize
    else:
        MDGrid = n_cells
    
    # Calculate IMGrid: number of cells to swap
    IMGrid = (n_cells * IM) // 100
    
    # Perform swaps
    for _ in range(IMGrid):
        if MDGrid == 0:
            break  # No swaps if MDGrid is 0
        # Randomly select first cell (i, j)
        i = np.random.randint(0, n_cells)
        j = np.random.randint(0, n_cells)
        
        # Randomly select second cell (mdI, mdJ) within MDGrid
        mdI = np.random.randint(0, MDGrid)
        mdJ = np.random.randint(0, MDGrid)
        while mdI == i and mdJ == j:  # Ensure different cells
            mdI = np.random.randint(0, MDGrid)
            mdJ = np.random.randint(0, MDGrid)
        
        # Find indices in DataFrame
        idx1 = new_df[(new_df['row'] == i) & (new_df['col'] == j)].index
        idx2 = new_df[(new_df['row'] == mdI) & (new_df['col'] == mdJ)].index
        
        if idx1.empty or idx2.empty:
            continue  # Skip if indices not found
        
        idx1, idx2 = idx1[0], idx2[0]
        
        # Swap all attributes except row and col
        columns_to_swap = ['status', 'learningCapability', 'transferCapability', 
                          'baseForgettingRate', 'baseQuittingRate', 'timesContactorD']
        temp = new_df.loc[idx1, columns_to_swap].copy()
        new_df.loc[idx1, columns_to_swap] = new_df.loc[idx2, columns_to_swap]
        new_df.loc[idx2, columns_to_swap] = temp
    
    return new_df    

In [ ]:
def neighbors(df, point, radius=2, neighbors_type='Moore'):
    row, col = point
    if not (0 <= row < n_cells and 0 <= col < n_cells):
        return []
    
    if neighbors_type not in ['Moore', 'VonNeumann']:
        raise ValueError("Vizinhança permitida Moore ou VonNeumann")
    
    # Calculate distances for all cells
    if neighbors_type == 'Moore':
        # Chebyshev distance: max(|row - row_i|, |col - col_i|)
        distances = np.maximum(np.abs(df['row'] - row), np.abs(df['col'] - col))
    else:
        # Manhattan distance: |row - row_i| + |col - col_i|
        distances = np.abs(df['row'] - row) + np.abs(df['col'] - col)
    
    # Filter cells within radius (including the center point)
    neighbor_indices = df[distances <= radius].index.tolist()
    
    return neighbor_indices